# Evaluating Taekwondo Moves
System to recognize and evaluate video of performed taekwondo moves

**Instructions**: Run all cells. When prompted to chose a file, first import `VideoPose3D-2021-02-23.zip` and next, `IdealSet_NPYs&MP4s.zip`.

###Get Existing Custom Datasets as of February 23, 2021 & Ideal Set Videos

In [ ]:
from google.colab import files
videopose3d = files.upload()

In [ ]:
unzip_folder(f'VideoPose3D-2021-02-23.zip')

In [ ]:
ideals = files.upload()

In [ ]:
unzip_folder('IdealSet_NPYs&MP4s.zip')

In [45]:
hknpz = files.upload()

KeyboardInterrupt: ignored

##Setup

In [1]:
import sys
import os
from os.path import exists, join, basename, splitext
import time
import math
import pickle
import numpy as np
import cv2
import json
import random
from google.colab.patches import cv2_imshow
from datetime import datetime

In [2]:
!pip install -U scikit-learn

     |████████████████████████████████| 22.3MB 1.9MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [3]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

In [4]:
def zip_folder(zip_folder_name, direc):
  !zip -q -r "$zip_folder_name" "$direc"

In [5]:
def unzip_folder(zip_folder_name, direc='/content'):
  !unzip "$zip_folder_name" -d "$direc"
  # !unzip -q -l "$zip_folder_name"

In [6]:
def remove_folder(folder_name):
  !rm -r "$folder_name"

In [7]:
def get_t():
  return f'{datetime.now()}'.replace(' ', '_').replace(':', '-')

In [8]:
#abbreviations to make folders 
abbreviations = {
    "High block": "hb",
    "Middle block (in-to-out)": "mbio",
    "Middle block (out-to-in)": "mboi",
    "Knife hand block": "khb",
    "Low block": "lb",
    "Punch (middle-level)": "p",
    "High punch (face-level)": "hp",
    "Front kick": "fk",
    "Round kick": "rk",
    "Side kick": "sk"
}

## Skeleton Representation

###Create Custom Datasets

In [ ]:
# git_repo_url = 'https://github.com/facebookresearch/VideoPose3D.git'
# project_name = splitext(basename(git_repo_url))[0]
# if not exists(project_name):
#   # clone and install dependencies
#    !git clone -q --depth 1 $git_repo_url

*Make sure to upload the pretrained model to the checkpoint folder if you took VideoPose3D straight from GitHub

In [11]:
!python3 -m pip install --upgrade pip

     |████████████████████████████████| 1.6MB 5.3MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [12]:
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7m
distutils: /usr/local/bin
sysconfig: /usr/bin
distutils: /usr/local
sysconfig: /usr
user = False
home = None
root = None
prefix = None
     |████████████████████████████████| 274 kB 5.7 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44074 sha256=ce7be184f8fc987b85bfde7bcb951b6b33f20c54b4ccb74d5970aa9f58db4386
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  distutils: /usr/local/lib/python3.7/dist-packages
  sysconfig: /usr/lib/python3.7/site-package

In [13]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.8)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
assert torch.__version__.startswith("1.8")   # need to manually install torch 1.8 if Colab changes its default version
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7m
distutils: /usr/local/bin
sysconfig: /usr/bin
distutils: /usr/local
sysconfig: /usr
user = False
home = None
root = None
prefix = None
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
     |████████████████████████████████| 6.2 MB 715 kB/s 
     |████████████████████████████████| 47 kB 3.8 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.3.post20210317-py3-none-any.whl size=58542 sha256=e76cfd2c4856bb706058c8005f6c9fafaba016436f01d1bda120d63cb4d1cd49
  Stored in directory: /root/.cache/pip/wheels/a6/02/09/10e3a0150eb92e5ecbee3677a813bffc32a8ec6f876bfe4adf
Successfully built fvcore
  distutils: /usr/local/lib/python3.7/dist-packages
  sysconfig: /usr/lib/python3.7/site-pac

In [14]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [15]:
def create_custom_dataset(leadup, ip_folder, slow):
  p = os.path.join(leadup, ip_folder)
  if slow == True:
    oup_folder = f'{ip_folder}_slow'
    new_p = os.path.join(leadup, oup_folder)
    if not os.path.exists(new_p):
      os.makedirs(new_p)

    for f in os.listdir(p):
      # print(f)
      vidfn, ext = os.path.splitext(f)
      slow_down_video(os.path.join(p, f), os.path.join(new_p, f), speed=2.0)
  else:
    new_p = p
    oup_folder = ip_folder

  twod_outputs_folder = f'{oup_folder}_2d_outputs'
  if not os.path.exists(twod_outputs_folder):
    os.makedirs(twod_outputs_folder)
  !cd VideoPose3D/inference && python3 infer_video_d2.py --cfg COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml --output-dir ../../"$twod_outputs_folder"  --image-ext mp4 ../../"$new_p"/
  !cd VideoPose3D/data/ && python3 prepare_data_2d_custom.py -i ../../"$twod_outputs_folder" -o "$oup_folder"_dataset
  return oup_folder, f'{oup_folder}_dataset'

## Pose Classification

In [16]:
def reconstruct_video_helper(path, threed_outputs_folder, custom_dataset, video_fn):
  p = os.path.join(path, video_fn)
  print(p)
  # print(f'cd VideoPose3D/ && python run.py -d custom -k {custom_dataset} -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject {video_fn}.mp4 --viz-action custom --viz-camera 0 --viz-video ../{p}.mp4 --viz-output ../{threed_outputs_folder}/{video_fn}_si.mp4 --viz-export ../{threed_outputs_folder}/{video_fn}_si --viz-size 6')
  !cd VideoPose3D/ && python run.py -d custom -k "$custom_dataset" -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject "$video_fn".mp4 --viz-action custom --viz-camera 0 --viz-video ../"$p".mp4 --viz-output ../"$threed_outputs_folder"/"$video_fn"_si.mp4 --viz-export ../"$threed_outputs_folder"/"$video_fn"_si --viz-size 6

In [17]:
def reconstruct_videos(leadup, prefix, dts, folder=None):
  # path = f'../{prefix}/'
  # print(f'Dataset name: {dts}')
  if folder is None:
    p = os.path.join(leadup, prefix)
    threed_outputs_folder = os.path.join(leadup, f'{prefix}_3d_outputs')
  else:
    p = os.path.join(leadup, folder)
    threed_outputs_folder = os.path.join(leadup, f'{folder}_3d_outputs')

  if not os.path.exists(threed_outputs_folder):
    os.makedirs(threed_outputs_folder)

  for f in os.listdir(p):
    print(f)
    vidfn, ext = os.path.splitext(f)
    reconstruct_video_helper(p, threed_outputs_folder, dts, vidfn)
  return threed_outputs_folder

###Generate Vocabulary Poses with K-means clustering

In [42]:
hand_techniques = ["High block", "Middle block (in-to-out)", "Middle block (out-to-in)", "Knife hand block", "Low block", "Punch (middle-level)", "High punch (face-level)"]
kicking_techniques = ["Front kick", "Round kick", "Side kick"]

hand_vp = 10
kick_vp = 12

ks_name = f'ks-h{hand_vp}k{kick_vp}.p'

In [19]:
from sklearn.cluster import KMeans

In [20]:
#to calculate angles between joints
#note that math returns angles in radians

h36m_num_joints = 17
h36m_joints = {
    0: [7,8,9],
    1: [14,8,11],
    2: [8,14,15],
    3: [14,15,16],
    4: [8,11,12],
    5: [11,12,13],
    6: [7,1,2],
    7: [1,2,3],
    8: [7,4,5],
    9: [4,5,6],
    10: [16,7,13],
    11: [3,7,6],
    12: [8,7,16],
    13: [8,7,13]
}

#note that the returned angle will be in radians
#where coordinates is a numpy array
def calc_angle(angle_jgroup, coordinates):
    angles = np.array([])
    for i in angle_jgroup:
        s,c,e = angle_jgroup[i]
        s_hat = coordinates[s] - coordinates[c]
        c_hat = coordinates[c] - coordinates[c]
        e_hat = coordinates[e] - coordinates[c]

        dot_product = s_hat@e_hat
        alpha = math.acos((dot_product)/(np.linalg.norm(s_hat)*np.linalg.norm(e_hat)))

        angles = np.append(angles, alpha)
    return angles

#given filename with frame.json, gets angles
def get_frame_angles(frame):
    ang = calc_angle(h36m_joints, frame)
    return ang

#rerturns a series of 14d arrays (angles in radians)
def get_angles(data):
    ang = np.empty((len(data),len(h36m_joints)))
    for i, d in enumerate(data):
        f_ang = get_frame_angles(d)
        ang[i] = f_ang
    return ang

In [21]:
#input - a bunch of NumPy archives of a given move
def get_cluster_centroids(k, ips):
  X = get_angles(np.load(ips[0]))
  i = 1
  while i < len(ips):
      data = get_angles(np.load(ips[i]))
      X = np.concatenate((X, data))
      i+=1
  # print(X.shape)
  kmeans = KMeans(n_clusters=k, random_state=0).fit(X)

  # print(kmeans.labels_)
  cc = kmeans.cluster_centers_
  # print(cc)
  return kmeans

In [22]:
def get_ips(leadup, prefix):
  threed_outputs_folder =  os.path.join(leadup, f'{prefix}_3d_outputs')
  ips = []
  for f in os.listdir(threed_outputs_folder):
      vidfn, ext = os.path.splitext(f)
      # print(ext)
      if ext == '.npy':
        ips.append(os.path.join(threed_outputs_folder, f))
  print(ips)
  return ips

###KNN

In [23]:
from sklearn.neighbors import KNeighborsClassifier

In [49]:
def classify_videos(ips, tech, ks_name, new_ids=False):
    path_to_ks = os.path.join(lu, ks_name)
    suffix = path_to_ks[path_to_ks.find('-')+1:path_to_ks.find('.p')]

    npzfile = np.load(os.path.join(lu, f'{suffix}.npz'))
    X,y = npzfile['arr_0'], npzfile['arr_1']
    identifier2move, move2identifier = pickle.load(open(os.path.join(lu, f'moveids.p'), 'rb'))

    # neigh = KNeighborsClassifier(n_neighbors=5)
    neigh = KNeighborsClassifier(n_neighbors=10)
    # neigh = KNeighborsClassifier(n_neighbors=12)
    neigh.fit(X, y)

    final_res = []
    for i, ip in enumerate(ips):
        if tech == None or tech[i] == 'k':
          num_vp = int(suffix[suffix.find('k')+1:])
        elif tech[i] == 'h':
            num_vp = int(suffix[suffix.find('h')+1:suffix.find('k')])

        sample_kmeans = get_cluster_centroids(num_vp, [ip])

        res = []
        for pose in sample_kmeans.cluster_centers_:
            res.append(neigh.predict([pose]))

        tally = {}
        for r in res:
            m = identifier2move[r[0]]
            if m not in tally:
                tally[m] = 1
            else:
                tally[m] += 1

        # for w in sorted(tally, key=tally.get, reverse=True):
        #     print(w, tally[w])
            # final_res.append(tally.values())
        maxvote_move = max(tally, key=tally.get)
        final_res.append((maxvote_move, f'{tally[maxvote_move]}/{sum(tally.values())}'))
    return final_res

## Temporal Alignment (DTW)

In [25]:
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

In [60]:
def dtw_compare_samples_helper(ips, oup):
  d = []
  y = get_angles(np.load(oup))
  for i in ips:
    X = get_angles(np.load(i))
    distance, path = fastdtw(X, y, dist=euclidean)
    d.append(distance)
  return d

In [59]:
def dtw_compare_samples(leadup, prefix, ips):
  oup = os.path.join(leadup, f'{prefix}_ideal_si.npy')
  d = dtw_compare_samples_helper(ips, oup)
  print(d)
  return d

## GUI

In [28]:
git_repo_url = 'https://github.com/SriMed/sr_eval_tkd_v2.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install dependencies
   !git clone -q --depth 1 $git_repo_url

In [29]:
!chmod u+x sr_eval_tkd_v2/convert2mp4.sh

In [47]:
lu = 'sr_eval_tkd_v2'

In [30]:
def convert2mp4(base):
  !cd "$base" && ./../"$lu"/convert2mp4.sh

In [54]:
from IPython.display import clear_output
def evaluate_video():
  f = files.upload()
  file = input('Enter the name of your video (ex. i/am/a/boss/test.mp4): ') #assume in directory
  lu_pipe, tail = os.path.split(file)
  vidfn, ext = os.path.splitext(tail)

  if ext != ".mp4":
    convert2mp4(lu_pipe)

  ip_f_path = os.path.join(lu_pipe, vidfn)
  if not os.path.exists(ip_f_path):
    os.makedirs(ip_f_path)
  !mv "$file" "$ip_f_path"
  
  oup_f, dts = create_custom_dataset(lu_pipe, vidfn, slow=False)
  threed_outputs_folder_path = reconstruct_videos(lu_pipe, None, dts, oup_f)
  show_local_mp4_video(f'{threed_outputs_folder_path}/{vidfn}_si.mp4')
  ips = get_ips(lu_pipe, oup_f)
  res = classify_videos(ips, None, ks_name)
  move = res[0][0]
  confidence = res[0][1]
  confidence = int(confidence[:confidence.find('/')])/int(confidence[confidence.find('/')+1:])
  
  clear_output()
  
  print(f"Your video was classified as a {move} with {confidence*100}% confidence")
  d = dtw_compare_samples('ideal_3d_outputs', move, ips)
  print(f"Relative to the exemplar, your score is {d}. The closer to 0, the better your execution. Try again to get closer to 0.")

In [64]:
evaluate_video()

Your video was classified as a fk_l with 33.33333333333333% confidence
[224.6556619163967]
Relative to the exemplar, your score is [224.6556619163967]. The closer to 0, the better your execution. Try again to get closer to 0.
